In [ ]:
import os
os.chdir('/')

import matplotlib
import numpy as np
import tensorflow as tf

import tfplot

from exch.scripts.sampler.Sampler import batchload
from exch.scripts.tensorflow_tiramisu.Tiramisu import Tiramisu

In [ ]:
iteration = str(67)

FILEDIR='/exch/gdrive/relevant_tiles'
boarddir_train = '/exch/scripts/tb_logs/train/'+iteration
boarddir_test = '/exch/scripts/tb_logs/test/'+iteration
SESSIONDIR = '/exch/scripts/tf_sess'
MODELFILE=SESSIONDIR+"/model"+iteration+".ckpt"

num_epochs = 10000
display_step = 2

In [ ]:
T = Tiramisu()

In [ ]:
X_batches, y_batches, xte, yte, MNs, SDs = batchload(FILEDIR, 0.8, int(8), int(3), int(32), False)
yte = np.expand_dims(np.array(yte), 3)

#X_batches = X_batches[:6]
#y_batches = y_batches[:6]

In [ ]:
train_writer = tf.summary.FileWriter(boarddir_train)
test_writer = tf.summary.FileWriter(boarddir_test)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    for e in range(1, num_epochs + 1):
        print('Epoch: ' + str(e))
        for i, xb in enumerate(X_batches):
            #print('Iteration: ' + str(i))
            yb = np.expand_dims(y_batches[i],3)
            sess.run(T.train_op, feed_dict={T.X: xb, T.y: yb, T.phase:True, T.kp:.8})
        
        if e % display_step == 0 or e == 1:
            print('val')
            summary, loss, acc = sess.run([T.merged, T.loss, T.acc], feed_dict={T.X: xb,
                                                                     T.y: yb,
                                                                     T.phase: False,
                                                                     T.kp: 1.})
            print("Training Accuracy: "+str(acc))
            train_writer.add_summary(summary, e)
            
            summary, loss, acc = sess.run([T.merged, T.loss, T.acc], feed_dict={T.X: xte,
                                          T.y: yte,
                                          T.phase: False,                                                                   
                                          T.kp: 1.})            
            print("Validation Accuracy: "+str(acc))
            test_writer.add_summary(summary, e)
            
            save_path = saver.save(sess, MODELFILE)
            print("Model saved in file: %s" % save_path)